# Session 4:  Post processing trajectories

<a id='trajandtrans'></a>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Authors: 

- Dr Micaela Matta - [@micaela-matta](https://github.com/micaela-matta)
- Dr Richard Gowers - [@richardjgowers](https://github.com/richardjgowers) 


### Learning outcomes 

- Understanding the treatment of periodic boundary conditions in MDAnalysis
- Using distance calculation methods to manipulate periodic images of molecules
- Creating your on-the-fly transformations using MDAnalysis


#### Additional resources

 - During the workshop, feel free to ask questions at any time
 - For more on how to use MDAnalysis, see the [User Guide](https://userguide.mdanalysis.org/2.0.0-dev0/) and [documentation](https://docs.mdanalysis.org/2.0.0-dev0/)
 - Ask questions on the [GitHub Discussions forum](https://github.com/MDAnalysis/mdanalysis/discussions) or on [Discord](https://discord.gg/fXTSfDJyxE)
 - Report bugs on [GitHub](https://github.com/MDAnalysis/mdanalysis/issues?)


# Google Colab package installs

This installs the necessary packages for Google Colab. Please only run these if you are using Colab.

In [ ]:
# NBVAL_SKIP
!pip install condacolab
import condacolab
condacolab.install()

In [ ]:
# NBVAL_SKIP
import condacolab
condacolab.check()
!mamba install -c conda-forge mdanalysis mdanalysistests mdanalysisdata nglview rdkit

In [ ]:
# NBVAL_SKIP
# enable third party jupyter widgets
from google.colab import output
output.enable_custom_widget_manager()

In [1]:
import warnings
warnings.filterwarnings("ignore")
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
import nglview as nv
%matplotlib inline

/Users/richard/mambaforge/envs/mda/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


## 1. Inspecting a loaded system

In this notebook we'll be looking at a protein-ligand complex.
One of the first things to do when loading a system is to inspect the data.
This is useful to check that we have loaded the correct data, and that the data is as expected.

In [2]:
u = mda.Universe('./data/topology.pdb', './data/traj.xtc')

Basic questions on loading a system into MDAnalysis:
- How many atoms are in the system?
- What residues are present?
- How many bonds are in the system?
- How many bonds should I expect in this system?

Our system seems correct, but the number of bonds seems low...
Let's try and figure out what went wrong.

In [7]:
# Based on observed resnames, selecting components of the system
protein = u.select_atoms('protein')
ligand = u.select_atoms('resname UNK')
ions = u.select_atoms('resname NA CL')

In [8]:
nv.show_mdanalysis(protein)

NGLWidget(max_frame=500)

Q: How could one verify that these three atom groups contain all of the atoms contained within u.atoms?

Q: How would we check the number of bonds in each of these different components of the system?


Q: How can bonds for an AtomGroup be guessed?

Q: Given the number of ions, approximately how many bonds should you expect?

Q: Given the ligand has 32 atoms, based on the number of bonds, can you tell how many rings are in the ligand molecule?

In [5]:
nv.show_mdanalysis(u.select_atoms('resname UNK'))

NGLWidget(max_frame=500)

Q: (hard) given the number of molecules and bonds in the entire system, how many rings are there in this system?


## 2. Periodic boundaries and minimising distances

Periodic boundary conditions allow for an approximation of an infinitely sized system by simulating a simple unit cell. This is illustrated below. The black box is the only cell we simulate; the tiled images around it are there for illustration. The green particle moves past the top boundary of the unit cell and are moved to the bottom of the box with the same velocity (illustrated by the red dashed line). This boundary condition keeps the volume and number of particles constant in the simulation.


<a href="https://upload.wikimedia.org/wikipedia/commons/2/2e/Limiteperiodicite.svg">
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/2e/Limiteperiodicite.svg" width="400"/a>

### Questions

Q: In a cubic box with box length L, what is the largest possible distance you can travel in a single dimension before wrapping back around on yourself?

A: L.  i.e. once you have travelled L in any single direction you are back to where you started.

Q: In a cubic box with length L, what is the largest possible separation between two objects along a single dimension?

A: L/2, any larger and there would be a smaller separation vector in the opposite direction.

Q: As above, but what is the largest possible separation for a 3 dimensional vector?

A: L/2 * sqrt(3)

In each dimension largest separation is L/2.
2D vector is sqrt(L^2/4 + L^2/4) = sqrt(L^2 / 2)
Then 3D is sqrt(L^2 / 2 + L^2 / 4) = sqrt(L^2 * 3/4) = L/2 sqrt(3)

### Advanced Questions

Q: In a rectangular box with sides [Lx, Ly, Lz] how do the above answers change?

Q: In a skewed triclinic box, how do the above answers change?

## 3. Periodic boundary treatment in MDAnalysis

The information on the periodic box is stored in the `Universe.dimensions` or `AtomGroup.dimensions` attribute.
This gives a description of the box as 6 values, the length in x, y, z dimensions and the angle between the three vectors.
For our example today:

In [ ]:
print(f'Our box is {u.dimensions}')

All angles between the vectors are 90 degrees, so the box is orthorhombic (rectangular).
Also the length of each dimension is identical, so we actually happen to have a cubic box.

In [ ]:
box_L = u.dimensions[0]
print(box_L)

### The `minimize_vectors` function

The `MDAnalysis.lib.distances.minimize_vectors` function will take an array (or single) 3D vector and box description and return the smallest version of these vectors (or vector).


Q: What is the expected return for minimize_vectors([L, L, L], u.dimensions) ?

Q: What is the expected return for minimize_vectors([1.2 * L, 0.2 L, 0.2 L], u.dimensions) ?

Q As above, for [0.6L, 0.2L, 0.2L]

Q: If the vector was the separation between two molecules, what does the **difference** between the original and minimised vector represent?

Q: If I translate a molecule by an entire box vector(s) in a given direction(s)f, has the molecule moved?

## 4. Checking the position of our ligand relative to the protein.

In our simulation we are expecting that the ligand molecule should stay close to its original position.
As a very simple check of our simulation we want to check the position of the ligand molecule relative to the protein molecule.  This will tell us if our simulation went as expected, or if the ligand was unhappy in its starting position and moved away.

In [ ]:
## Guide for how to solve the above question step by step?
# For a single frame

# How do we get the center of mass of an AtomGroup?

# How do we find the distance between two coordinates (i.e. centers of mass)

In [ ]:

# Building this up, how would we iterate over a trajectory and calculate this for each frame?

# How would we allocate an array to store these distances and store each frame in this array?

In [ ]:
fig, ax = plt.subplots()

ax.plot(distance)

In [ ]:
nv.show_mdanalysis(u.tao

### Co-locating the ligand and protein

Although apparently separate, we have a hunch that our ligand and protein might actually be closer than they initially appear...

In [ ]:
 distance[0] / box_L

Q: Calculate the vector separating the protein COM from the ligand COM

In [ ]:
separation = protein.center_of_mass() - ligand.center_of_mass()

We can then calculate the minimised version of this vector.

In [ ]:
minimal_separation = mda.lib.distances.minimize_vectors(separation, box=u.dimensions)

And calculate the difference between these "raw" and "minimised" vectors:

In [ ]:
separation - minimal_separation

In [ ]:
(separation - minimal_separation) / box_L

The `AtomGroup.translate` method allows the positions of `atoms inside this group to be shifted

In [ ]:
ligand.translate(separation - minimal_separation)

The separation vector is now much smaller than before

In [ ]:
protein.center_of_mass() - ligand.center_of_mass()

## Writing a function to shift the ligand to the closest image to the protein

Using all of the above working, can you write a function that for each timestep,
translates the ligand molecule to move it into the closest image with the protein each step?

In [ ]:
# some skeleton code...
def reimage_ligand(protein, ligand):
    # calculate a separation vector...

    # figure out a shift in box lengths to apply to the ligand

    # translate the ligand in-place
    pass

for ts in u.trajectory:
    reimage_ligand(protein, ligand)

In [ ]:
# Q: which periodic image of the ligand is correct?
# a: trick question, all images are correct, depending on the calculation
#    for looking at ligand interaction with protein, likely the one that minimises COM distance is correct

## 5. Putting it all together - Transformations

MDAnalysis contains a system for automatically applying these sorts of post-simulation modifications to trajectories called `transformations`.
These are one or more functions which alter the trajectory data just after it is loaded,
and are applied automatically whenever the trajectory is manipulated (i.e. iterated or seeked etc).

We can write our own `Transformation` objects to apply the ligand re-imaging automatically.
To do this we must import the `TransformationBase` class, and create a subclass which we will call `LigandShift`.

There are two methods (functions on the class) that we must write.
First the `__init__` method defines what information the transformation requires to operate,
in our case this is the definition of the protein and ligand AtomGroups.
Next the `._transform` method defines the mathematical operations that will take place which will operate on these two AtomGroups.

Using the previously written functions, finish the `LigandShift` transformation class.

In [ ]:
from MDAnalysis.transformations.base import TransformationBase

class LigandShift(TransformationBase):
    def __init__(self, protein, ligand):
        super().__init__()
        self.protein = protein
        self.ligand = ligand

    def _transform(self, ts):
        # put ligand shifting code here
        
        return ts

In [ ]:
u2 = mda.Universe('./data/topology.pdb', './data/traj.xtc')

prot2 = u2.select_atoms('protein')
ligand2 = u2.select_atoms('resname UNK')

In [ ]:
ligshift = LigandShift(prot2, ligand2)

In [ ]:
u2.trajectory.add_transformations(ligshift)

In [ ]:
# run the above analysis wrt ligand-protein position
# and check that the transformation pipeline has properly post-processed your trajectory data

## 6. Extension work

Transformations are a powerful way to transform trajectory data as it is loaded.
There are many transformation modules built in to MDAnalysis that perform useful operations such as
re-imaging molecules, unwrapping or wrapping around periodic boundaries and rotating and aligning molecules.
